### 1. Data Preprocessing

In [66]:
import pandas as pd

In [67]:
# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [68]:
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [69]:
train['Activity'].unique()

array(['STANDING', 'SITTING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'], dtype=object)

In [70]:
train.isnull().sum()

tBodyAcc-mean()-X       0
tBodyAcc-mean()-Y       0
tBodyAcc-mean()-Z       0
tBodyAcc-std()-X        0
tBodyAcc-std()-Y        0
                       ..
angle(X,gravityMean)    0
angle(Y,gravityMean)    0
angle(Z,gravityMean)    0
subject                 0
Activity                0
Length: 563, dtype: int64

In [71]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['Activity'] = le.fit_transform(train['Activity'])
test['Activity'] = le.transform(test['Activity'])

In [72]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(train.drop('Activity', axis=1))
X_test = scaler.transform(test.drop('Activity', axis=1))
y_train = train['Activity']
y_test = test['Activity']

In [73]:
X_train

array([[ 0.20064157, -0.0636826 , -0.41962845, ...,  0.40794614,
        -0.00756789, -1.8288513 ],
       [ 0.05594788,  0.03148567, -0.25390836, ...,  0.40911698,
         0.00787517, -1.8288513 ],
       [ 0.07351535, -0.04341648, -0.07629468, ...,  0.4102883 ,
         0.02650234, -1.8288513 ],
       ...,
       [-0.01566765,  0.0167814 ,  1.13222107, ...,  0.64059683,
         0.34870928,  1.4025149 ],
       [ 0.21586648, -0.02812252, -0.86770988, ...,  0.63147758,
         0.29327564,  1.4025149 ],
       [ 1.09620157,  0.12919873, -1.67268082, ...,  0.63274259,
         0.33396081,  1.4025149 ]])

In [74]:
y_train

0       2
1       2
2       2
3       2
4       2
       ..
7347    5
7348    5
7349    5
7350    5
7351    5
Name: Activity, Length: 7352, dtype: int64

In [75]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [76]:
# Create train dataset
train_dataset = CustomDataset(X_train, y_train)

In [77]:
# Create test dataset
test_dataset = CustomDataset(X_test, y_test)

### 2. Create a base model

In [78]:
import torch.nn as nn

class MyNN(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate):
        super().__init__()

        layers = []

        for i in range(num_hidden_layers):
            layers.append(nn.Linear(input_dim, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = neurons_per_layer

        layers.append(nn.Linear(neurons_per_layer, output_dim))

        self.model = nn.Sequential(*layers) # unpacking the layers as sequential requires individual layers
    
    def forward(self, x):
        return self.model(x)

### 3. Hyperparameter Tuning

In [79]:
import torch.optim as optim
from torch.utils.data import DataLoader

# objective function
def objective(trial):
    # next hyperparameter value from search space
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step=8)
    epochs = trial.suggest_int("epochs", 10, 50, step = 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log = True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step = 0.1)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log = True)

    # Create train and test loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # model init
    input_dim = X_train.shape[1]
    output_dim = len(y_train.unique())

    model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)

    # define loss 
    criterion = nn.CrossEntropyLoss()

    # define optimizer
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    if optimizer_name == 'Adam':
        optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'RMSprop':
        optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # training loop
    for epoch in range(epochs):

        for batch_features, batch_labels in train_loader:

            # forward pass
            outputs = model(batch_features)

            # calculate loss
            loss = criterion(outputs, batch_labels)

            # back pass
            optimizer.zero_grad()
            loss.backward()

            # update grads
            optimizer.step()

    # evaluation on test data
    total = 0
    correct = 0

    with torch.no_grad(): 

        for batch_features, batch_labels in test_loader:

            outputs = model(batch_features)

            _, predicted = torch.max(outputs, 1)

            total = total + batch_labels.shape[0]

            correct = correct + (predicted == batch_labels).sum().item()

        accuracy = (correct/total)    

    return accuracy

In [80]:
import optuna 

study = optuna.create_study(direction = 'maximize')

study.optimize(objective, n_trials = 10)

[I 2025-03-12 23:15:06,673] A new study created in memory with name: no-name-a05ebadd-05da-4bd6-98cf-ab35c8c85c07
[I 2025-03-12 23:15:13,393] Trial 0 finished with value: 0.6973193077706142 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 96, 'epochs': 20, 'learning_rate': 0.0022774907320188887, 'dropout_rate': 0.30000000000000004, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 0.0007488425036176591}. Best is trial 0 with value: 0.6973193077706142.
[I 2025-03-12 23:15:19,213] Trial 1 finished with value: 0.3013233797081778 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 96, 'epochs': 10, 'learning_rate': 0.0012667641437199984, 'dropout_rate': 0.5, 'batch_size': 16, 'optimizer': 'Adam', 'weight_decay': 2.1420715862869445e-05}. Best is trial 0 with value: 0.6973193077706142.
[I 2025-03-12 23:15:22,956] Trial 2 finished with value: 0.24363759755683745 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 104, 'epochs': 10, 'learning_rate': 0.000

In [86]:
study.best_value

0.8941296233457754

In [87]:
study.best_params

{'num_hidden_layers': 4,
 'neurons_per_layer': 128,
 'epochs': 30,
 'learning_rate': 0.024829391892815085,
 'dropout_rate': 0.4,
 'batch_size': 128,
 'optimizer': 'RMSprop',
 'weight_decay': 0.00020729961329215226}

### 4. Train the model with best params

In [83]:
# Creating DataLoader with Tuned Batch Size
from torch.utils.data import DataLoader

batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [88]:
# Initialize model with tuned hyperparameters
input_dim = X_train.shape[1]
output_dim = len(y_train.unique())

model = MyNN(
    input_dim=input_dim,
    output_dim=output_dim,
    num_hidden_layers=4,
    neurons_per_layer=128,
    dropout_rate=0.4
)

# Set optimizer and loss function
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0248, weight_decay=0.000207)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/30, Loss: 1.0559
Epoch 2/30, Loss: 0.4747
Epoch 3/30, Loss: 0.3777
Epoch 4/30, Loss: 0.3300
Epoch 5/30, Loss: 0.3443
Epoch 6/30, Loss: 0.3092
Epoch 7/30, Loss: 0.3226
Epoch 8/30, Loss: 0.3263
Epoch 9/30, Loss: 0.2951
Epoch 10/30, Loss: 0.2977
Epoch 11/30, Loss: 0.3329
Epoch 12/30, Loss: 0.2983
Epoch 13/30, Loss: 0.3108
Epoch 14/30, Loss: 0.3218
Epoch 15/30, Loss: 0.3308
Epoch 16/30, Loss: 0.2613
Epoch 17/30, Loss: 0.3486
Epoch 18/30, Loss: 0.3044
Epoch 19/30, Loss: 0.2962
Epoch 20/30, Loss: 0.3288
Epoch 21/30, Loss: 0.3411
Epoch 22/30, Loss: 0.3394
Epoch 23/30, Loss: 0.3094
Epoch 24/30, Loss: 0.3042
Epoch 25/30, Loss: 0.3337
Epoch 26/30, Loss: 0.3526
Epoch 27/30, Loss: 0.3388
Epoch 28/30, Loss: 0.3188
Epoch 29/30, Loss: 0.2995
Epoch 30/30, Loss: 0.3245


In [89]:
model.eval()
total, correct = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

train_accuracy = correct / total
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

Test Accuracy: 86.80%
Training Accuracy: 89.30%
